In [ ]:
from __future__ import print_function
import math
import numpy
from scipy.spatial import Voronoi, voronoi_plot_2d, cKDTree

In [ ]:
from ufl import *
import dune.ufl
import dune.fem
import dune.fem.function as gf
from dune.fem.plotting import plotPointData as plot
import dune.create as create

def error(grid,df, exact):
    df_coeff = dune.ufl.GridCoefficient(df)
    l2error_gf = create.function("ufl", grid, "error", 5,
            as_vector([(exact[0]-df_coeff[0])**2]) )
    return math.sqrt( l2error_gf.integrate() )

The actual agnglomerated method:
initialize with the number of voronoi cells to use. The points will be distributed randomly. 

In [ ]:
# http://zderadicka.eu/voronoi-diagrams/
from voronoi import triangulated_voronoi
class Agglomerate:
    def __init__(self,N):
        self.ind = set()
        self.N = N
        self.suffix = str(self.N)
        numpy.random.seed(1234)
        self.voronoi_points = numpy.random.rand(self.N, 2)
        self.voronoi_kdtree = cKDTree(self.voronoi_points)
        vor = Voronoi(self.voronoi_points)
        voronoi_plot_2d(vor).savefig("agglomerate_voronoi"+self.suffix+".pdf", bbox_inches='tight')
    def __call__(self,en):
        p = en.geometry.center
        test_point_dist, test_point_regions = self.voronoi_kdtree.query([p], k=1)
        index = test_point_regions[0]
        self.ind.add(index)
        return index
    def check(self):
     return len(self.ind)==self.N

The method for solving a non linear elliptic problem. Arguments are
- grid: the fine grid
- agglomerate: a possible agglomeration for the grid
- model: the elliptic model to solve
- exact: the exact solution used for computing the error
- name: a string to use for output
- space: the name of the space to ue, e.g., Lagrange or Vem
- scheme: the name of the scheme to use, e.g., h1 or Vem

Returns the discrete solution and an interpolation of the exact solution.

In [ ]:
def solve(grid,agglomerate,model,exact,name,space,scheme,penalty=None):
    print("SOLVING: ",name)
    gf_exact = create.function("ufl",grid,"exact",4,exact)
    if agglomerate:
        print("using agglomerate...")
        spc = create.space(space, grid, agglomerate, dimrange=1, order=1, storage="fem")
        # assert agglomerate.check(), "missing or too many indices provided by agglomoration object. Should be "+str()+" was "+str(len(ind))
    else:
        spc = create.space(space, grid, dimrange=1, order=1, storage="fem")
    interpol = spc.interpolate( gf_exact, "exact_"+name )
    if penalty:
        df,info = create.scheme(scheme, spc, model, penalty).solve(name=name)
    else:
        df,info = create.scheme(scheme, spc, model).solve(name=name)
    print(name+" size:",spc.size,"L2-error:", error(grid,df,exact), error(grid,interpol,exact),\
          info["linear_iterations"], info["iterations"])
    plot(df)
    return interpol, df

The actual algorithm taking the agglomeration object and solving on a (agglomeratied) simplex grid with both conforming FE and VEM.

In [ ]:
def compute(agglomerate):
    bounding_box = numpy.array([0., 1., 0., 1.]) # [x_min, x_max, y_min, y_max]
    points, triangles = triangulated_voronoi(agglomerate.voronoi_points, bounding_box)
    grid = create.grid("ALUSimplex", {'vertices':points, 'simplices':triangles}, dimgrid=2)

    print("grid size:",grid.size(0))
        
    uflSpace = dune.ufl.Space((grid.dimGrid, grid.dimWorld), 1, field="double")
    u = TrialFunction(uflSpace)
    v = TestFunction(uflSpace)
    x = SpatialCoordinate(uflSpace.cell())
    exact = as_vector( [cos(2.*pi*x[0])*cos(2.*pi*x[1])] )
    a = (inner(grad(u), grad(v)) + inner(u,v)) * dx
    a = a + 20./(u[0]*u[0]+1.) * v[0] * dx
    model = create.model("elliptic", grid, a==0, exact=exact ) # , dirichlet={ 1:exact } )

    interpol_lag, df_lag = solve(grid,None,       model,exact,"h1","Lagrange","h1")
    interpol_vem, df_vem = solve(grid,agglomerate,model,exact,"vem","AgglomeratedVEM","vem")

    grid.writeVTK("agglomerate"+agglomerate.suffix,
            pointdata=[ df_vem, interpol_vem ],
            celldata =[ create.function("local",grid,"cells",1,lambda en,x: [agglomerate(en)]) ])

In [ ]:
dune.fem.parameter.append("../data/parameter")
compute(Agglomerate(71))